# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
print(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [3]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:100]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate),
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [4]:
small_tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 128
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 32
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 32
    })
})

In [6]:
small_imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 128
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
})

In [5]:
small_tokenized_dataset["train"]["text"][:10]

["I didn't know this was a silent movie with narration. I don't care for silent movies - the corny humor, flickering lighting and film, etc. I'm sure that attributes to the low score I assigned it. It was about chapter 8 before I found any interest in this story and had I had popcorn I may have thrown it at the screen. Maybe this appeals to the sci-fi crowd? The only thing missing was a zombie scene and a brain transplant. I went with two other people on a Friday night and there were a total of 6 people in",
 'Well, first off, if you\'re checking out Revolt of the Zombies as some very early Night of the Living Dead (1968)-type film, forget it. This is about "zombies" in a more psychological sense, where that term merely denotes someone who is not in control of their will, but who must instead follow the will of another. The "zombies" here, as little as they are in the film, are largely metaphors for subservience to the state or authority in general, as in wartime. It is quite a stretch

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [7]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=7,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.697200,0.691796,0.468750
2,0.660900,0.684593,0.468750
3,0.626600,0.647037,0.687500
4,0.528400,0.585794,0.812500
5,0.392900,0.544018,0.781250
6,0.279000,0.472537,0.781250
7,0.206700,0.465684,0.812500


TrainOutput(global_step=56, training_loss=0.4845269535269056, metrics={'train_runtime': 1423.4034, 'train_samples_per_second': 0.629, 'train_steps_per_second': 0.039, 'total_flos': 44720165865024.0, 'train_loss': 0.4845269535269056, 'epoch': 7.0})

In [9]:
results = trainer.predict(small_tokenized_dataset['val'])

print(results)

PredictionOutput(predictions=array([[ 8.13814878e-01, -6.13799632e-01],
       [-9.20512974e-01,  1.53448820e+00],
       [-4.76370156e-01,  1.00458491e+00],
       [-5.48030972e-01,  1.09845543e+00],
       [ 3.30197185e-01,  3.16814691e-01],
       [-6.44745231e-01,  1.12002969e+00],
       [ 9.72069800e-01, -5.33408821e-01],
       [ 9.33605373e-01, -7.85885274e-01],
       [ 4.03095901e-01,  1.60675243e-01],
       [ 5.96987367e-01, -3.44549775e-01],
       [ 1.81460202e-01,  4.68340993e-01],
       [ 6.38225913e-01, -3.14653009e-01],
       [ 7.63136387e-01, -2.43683249e-01],
       [-8.23276997e-01,  1.30692554e+00],
       [-6.27229512e-01,  1.10227060e+00],
       [ 1.64988846e-01,  2.21196175e-01],
       [ 7.97619224e-01, -3.66945654e-01],
       [ 7.86765218e-01, -4.66903925e-01],
       [-4.24098819e-01,  1.11745584e+00],
       [ 4.28029895e-01, -1.12339854e-04],
       [ 5.35960309e-02,  6.09925389e-01],
       [ 1.53443828e-01,  5.04079342e-01],
       [ 3.89643490e-01, 

In [11]:
import torch

In [12]:
val_set = small_tokenized_dataset["val"]["text"]

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-56")
tokenizer = AutoTokenizer.from_pretrained("sample_cl_trainer/checkpoint-32")

model_inputs = tokenizer(val_set, return_tensors="pt", padding=True, truncation=True)
model_outputs = fine_tuned_model(**model_inputs)

predictions = torch.softmax(model_outputs.logits, dim=1)
predicted_indices = torch.argmax(predictions, dim=1)

labels = ["NEGATIVE", "POSITIVE"]
predicted_labels = [labels[idx] for idx in predicted_indices]
print(predicted_labels)

['NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE']


For deeper understanding of elements in the code

In [13]:
print(val_set)

print(len(val_set))

["It's a good thing I didn't watch this while i was pregnant.I definitely would have cried my eyes out and/or vomit. It was Kind of gruesome mainly disturbing. I personally thought the baby was adorable in its own twisted little way.However as a mom I cringed when Beth stabbed herself in the stomach and when Virgina aborted the child during her 3rd trimester with rusty utensils no less.Also,as an animal lover i almost cried when she scratched the cat to a bloody pulp.However,As creepy and sinister as the baby was I was rooting for it to live.And as twisted as", 'I have been a fan of Pushing Daisies since the very beginning. It is wonderfully thought up, and Bryan Fuller has the most remarkable ideas for this show.<br /><br />It is unbelievable on how much TV has been needing a creative, original show like Pushing Daisies. It is a huge relief to see a show, that is unlike the rest, where as, if you compared it to some of the newer shows, such as Scrubs and House, you would see the simil

In [14]:
print(model_outputs.logits.shape)

torch.Size([32, 2])


In [15]:
print(model_inputs)

print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=True))

{'input_ids': tensor([[ 101, 1135,  112,  ...,    0,    0,    0],
        [ 101,  146, 1138,  ...,    0,    0,    0],
        [ 101,  119,  119,  ...,    0,    0,    0],
        ...,
        [ 101, 1987,  119,  ...,    0,    0,    0],
        [ 101, 5203, 1136,  ...,    0,    0,    0],
        [ 101, 1192, 1221,  ..., 1194, 1126,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}
["It ' s a good thing I didn ' t watch this while i was pregnant. I definitely would have cried my eyes out and / or vomit. It was Kind of gruesome mainly disturbing. I personally thought the baby was adorable in its own twisted little way. However as a mom I cringed when Beth stabbed herself in the stomach and when Virgina aborted the child during her 3rd trimester with rusty utensils no less. Also, as an animal lover

In [16]:
print(predictions)

print(len(predictions))

tensor([[0.8065, 0.1935],
        [0.0791, 0.9209],
        [0.1853, 0.8147],
        [0.1616, 0.8384],
        [0.5033, 0.4967],
        [0.1462, 0.8538],
        [0.8184, 0.1816],
        [0.8481, 0.1519],
        [0.5603, 0.4397],
        [0.7194, 0.2806],
        [0.4288, 0.5712],
        [0.7217, 0.2783],
        [0.7324, 0.2676],
        [0.1062, 0.8938],
        [0.1507, 0.8493],
        [0.4860, 0.5140],
        [0.7622, 0.2378],
        [0.7779, 0.2221],
        [0.1763, 0.8237],
        [0.6054, 0.3946],
        [0.3644, 0.6356],
        [0.4132, 0.5868],
        [0.5686, 0.4314],
        [0.8044, 0.1956],
        [0.2563, 0.7437],
        [0.7769, 0.2231],
        [0.6961, 0.3039],
        [0.2898, 0.7102],
        [0.7298, 0.2702],
        [0.3716, 0.6284],
        [0.1133, 0.8867],
        [0.5066, 0.4934]], grad_fn=<SoftmaxBackward0>)
32


👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [17]:
trainer_test = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [18]:
trainer_test.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.210000,0.752263,0.625000
2,0.055400,0.703849,0.718750
3,0.020900,0.801823,0.750000
4,0.007500,0.951504,0.750000
5,0.005300,0.967533,0.750000
6,0.003800,0.972275,0.750000
7,0.003500,0.979785,0.750000


TrainOutput(global_step=56, training_loss=0.04376880202575454, metrics={'train_runtime': 1334.0106, 'train_samples_per_second': 0.672, 'train_steps_per_second': 0.042, 'total_flos': 44720165865024.0, 'train_loss': 0.04376880202575454, 'epoch': 7.0})

In [19]:
results = trainer_test.predict(small_tokenized_dataset['test'])

print(results)

PredictionOutput(predictions=array([[ 1.6943945 , -1.5095538 ],
       [ 0.28908676,  0.07643783],
       [-0.48212227,  1.2773657 ],
       [ 1.6632584 , -1.101342  ],
       [ 1.4405348 , -1.126184  ],
       [-1.216704  ,  2.080921  ],
       [-1.4746464 ,  2.4225748 ],
       [ 0.7948476 , -0.35011616],
       [-1.5311568 ,  2.4769602 ],
       [-0.48960018,  0.89745367],
       [ 1.3752024 , -1.2943267 ],
       [-1.0984235 ,  1.8014097 ],
       [ 1.7727002 , -1.3648314 ],
       [ 1.0739326 , -0.70289475],
       [ 0.7466451 , -0.4183288 ],
       [-1.802526  ,  2.6578512 ],
       [-1.8234149 ,  2.6482506 ],
       [-1.1486772 ,  1.5695202 ],
       [ 1.419614  , -0.8866112 ],
       [ 1.2603185 , -1.0896726 ],
       [-0.50896394,  1.3351524 ],
       [-1.2279835 ,  2.0912883 ],
       [ 1.5816946 , -1.2471101 ],
       [ 0.57132727, -0.29264256],
       [-1.3225197 ,  2.1884727 ],
       [-1.5714283 ,  2.346786  ],
       [ 1.950026  , -1.6307518 ],
       [-1.3538642 ,  2.30

In [20]:
test_str = "I love this movie!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-24")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

POSITIVE


In [21]:
test_set = small_tokenized_dataset["test"]["text"]

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-24")
tokenizer = AutoTokenizer.from_pretrained("sample_cl_trainer/checkpoint-32")

model_inputs = tokenizer(test_set, return_tensors="pt", padding=True, truncation=True)
model_outputs = fine_tuned_model(**model_inputs)

predictions = torch.softmax(model_outputs.logits, dim=1)
predicted_indices = torch.argmax(predictions, dim=1)

labels = ["NEGATIVE", "POSITIVE"]
predicted_labels = [labels[idx] for idx in predicted_indices]
print(predicted_labels)

['NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE']
